In [35]:
# Import libraries and dependencies
import os
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import datetime 

%matplotlib inline

In [23]:
# Load .env enviroment variables
from dotenv import load_dotenv
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create an object for aplpaca api 
api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)


In [24]:
sectors_csv = Path("Resources/spy_etf_sectors.csv")
sectors_df = pd.read_csv(sectors_csv)

sectors_df = sectors_df.drop(columns=["Zacks\nG Sector", "\nCompany", "% of\nPortfolio"])

sectors_df = sectors_df.rename(columns={
    "\nTicker": "ticker",
    "Russell\nSector": "sector"
})

,\nCompany,\nTicker,% of\nPortfolio,Russell\nSector,Zacks\nG Sector
0,ACTIVISION BLZD,ATVI,0.2%,TECHNOLOGY,COMMUNICATION SVCS
1,ALPHABET INC-A,GOOGL,1.6%,TECHNOLOGY,COMMUNICATION SVCS
2,ALPHABET INC-C,GOOG,1.6%,TECHNOLOGY,COMMUNICATION SVCS
3,AT&T INC,T,0.7%,UTILITIES,COMMUNICATION SVCS
4,CENTURYLINK INC,CTL,0.0%,UTILITIES,COMMUNICATION SVCS
...,...,...,...,...,...
499,PUBLIC SV ENTRP,PEG,0.1%,UTILITIES,UTILITIES
500,SEMPRA ENERGY,SRE,0.1%,UTILITIES,UTILITIES
501,SOUTHERN CO,SO,0.2%,UTILITIES,UTILITIES
502,WEC ENERGY GRP,WEC,0.1%,UTILITIES,UTILITIES


In [25]:
dxy_csv = Path("Resources/$dxy_historical_data_01-01_to_09-15-2020.csv")
dxy_df = pd.read_csv(dxy_csv)
dxy_df["ticker"]="DXY"
dxy_df = dxy_df.rename(columns={"Last":"close", "Time":"date"})
dxy_df = dxy_df[["date", "close", "ticker"]].dropna()

In [26]:
btc_csv = Path("Resources/btcusd_historical_data_01-01_to_09-15-2020.csv")
btc_df = pd.read_csv(btc_csv)
btc_df["ticker"]="BTC"
btc_df = btc_df.rename(columns={"Last":"close", "Time":"date"}).dropna()
btc_df = btc_df[["date", "close", "ticker"]]

In [27]:
xau_csv = Path("Resources/btcusd_historical_data_01-01_to_09-15-2020.csv")
xau_df = pd.read_csv(xau_csv)
xau_df["ticker"]="XAU"
xau_df = xau_df.rename(columns={"Last":"close", "Time":"date"}).dropna()
xau_df = xau_df[["date", "close", "ticker"]]

In [28]:
alternative_assets = pd.concat([dxy_df, btc_df, xau_df])
alternative_assets

,date,close,ticker
0,9/15/2020,93.07,DXY
1,9/14/2020,93.05,DXY
2,9/11/2020,93.27,DXY
3,9/10/2020,93.34,DXY
4,9/9/2020,93.26,DXY
...,...,...,...
180,1/7/2020,8018.82,XAU
181,1/6/2020,7580.60,XAU
182,1/3/2020,7278.13,XAU
183,1/2/2020,6974.40,XAU


In [44]:
# Set the ticker to stripes of 100
ticker_list = sectors_df["ticker"].to_list()
ticker_stripe_1 = ticker_list[0:99]
ticker_stripe_2 = ticker_list[99:198]
ticker_stripe_3 = ticker_list[198:297]
ticker_stripe_4 = ticker_list[297:396]
ticker_stripe_5 = ticker_list[396:495]
ticker_stripe_6 = ticker_list[495:]

# Set timeframe to '1D'
timeframe = "1D"

# Set start and end datetimes of from Jan 1 2020 to Sep 15 2020
start_date = start = pd.Timestamp("2020-01-01", tz="America/Chicago").isoformat()
end_date = pd.Timestamp("2020-09-15", tz="America/Chicago").isoformat()

# Chunk the data by stripes to make it compaitible with alpaca api ingestion 

spy_stripe_1 = api.get_barset(
    ticker_stripe_1,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_2 = api.get_barset(
    ticker_stripe_2,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_3 = api.get_barset(
    ticker_stripe_3,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_4 = api.get_barset(
    ticker_stripe_4,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_5 = api.get_barset(
    ticker_stripe_5,
    timeframe,
    start=start_date,
    end=end_date
).df

spy_stripe_6 = api.get_barset(
    ticker_stripe_6,
    timeframe,
    start=start_date,
    end=end_date
).df

# Combine all of the stripes together 
spy_df = pd.concat([spy_stripe_1, spy_stripe_2, spy_stripe_3, spy_stripe_4, spy_stripe_5, spy_stripe_6], axis=1)
spy_df

AAP                                        ADM  \
                             open     high       low   close   volume   open   
2020-01-02 00:00:00-05:00  160.47  160.950  157.5300  159.40   726650  46.57   
2020-01-03 00:00:00-05:00  158.03  159.630  157.2500  159.48   482371  45.89   
2020-01-06 00:00:00-05:00  157.74  158.890  156.4325  156.82   571689  45.79   
2020-01-07 00:00:00-05:00  156.90  157.400  152.4090  154.95  1021104  45.57   
2020-01-08 00:00:00-05:00  154.85  156.050  153.1400  153.15   706120  45.11   
...                           ...      ...       ...     ...      ...    ...   
2020-09-09 00:00:00-04:00  152.62  155.680  152.6200  154.30  1040694  45.51   
2020-09-10 00:00:00-04:00  155.53  155.550  153.1050  153.57   650622  46.47   
2020-09-11 00:00:00-04:00  154.47  155.320  153.0800  154.56   879257  46.13   
2020-09-14 00:00:00-04:00  156.90  158.915  156.1300  157.35  1322800  47.00   
2020-09-15 00:00:00-04:00  157.89  158.410  156.5100  156.89   415068  47.78   

                                                            ...     WEC  \
                             high      low  close   volume  ...    open   
2020-01-02 00:00:00-05:00  46.640  45.8800  46.11  1899517  ...  92.400   
2020-01-03 00:00:00-05:00  46.190  45.7200  46.02  1512616  ...  90.650   
2020-01-06 00:00:00-05:00  45.845  45.3200  45.67  2045946  ...  91.340   
2020-01-07 00:00:00-05:00  45.690  45.1000  45.11  1567481  ...  90.890   
2020-01-08 00:00:00-05:00  45.190  44.3800  44.61  2640257  ...  91.210   
...                           ...      ...    ...      ...  ...     ...   
2020-09-09 00:00:00-04:00  46.340  45.4700  46.06  5600146  ...  96.300   
2020-09-10 00:00:00-04:00  46.590  45.9386  46.02  4053865  ...  97.275   
2020-09-11 00:00:00-04:00  46.970  46.0100  46.89  5603942  ...  96.110   
2020-09-14 00:00:00-04:00  47.760  46.9600  47.50  5657532  ...  96.550   
2020-09-15 00:00:00-04:00  47.900  47.2700  47.47  2027878  ...  97.750   

                                                                  XEL  \
                              high     low  close     volume     open   
2020-01-02 00:00:00-05:00  92.5200  90.455  90.68  1267002.0  63.5500   
2020-01-03 00:00:00-05:00  91.7100  90.520  91.22  1405515.0  62.2800   
2020-01-06 00:00:00-05:00  91.8000  90.800  91.34   907752.0  62.7600   
2020-01-07 00:00:00-05:00  91.0700  90.340  90.96   930917.0  62.1500   
2020-01-08 00:00:00-05:00  91.3850  90.510  91.11   806742.0  62.5900   
...                            ...     ...    ...        ...      ...   
2020-09-09 00:00:00-04:00  99.0400  95.970  97.76  1872756.0  69.1400   
2020-09-10 00:00:00-04:00  97.6700  95.860  95.96  1707418.0  70.2400   
2020-09-11 00:00:00-04:00  96.3000  94.930  95.92  1619722.0  69.2500   
2020-09-14 00:00:00-04:00  97.7717  95.830  97.14  1169398.0  68.4268   
2020-09-15 00:00:00-04:00  99.2800  97.750  99.18   866768.0  69.8600   

                                                              
                              high      low   close   volume  
2020-01-02 00:00:00-05:00  63.5800  62.2000  62.380  2340833  
2020-01-03 00:00:00-05:00  62.9150  62.2800  62.680  1908930  
2020-01-06 00:00:00-05:00  62.9400  62.3300  62.590  1392103  
2020-01-07 00:00:00-05:00  62.5168  61.9700  62.460  1620406  
2020-01-08 00:00:00-05:00  62.6400  62.1200  62.420  1630637  
...                            ...      ...     ...      ...  
2020-09-09 00:00:00-04:00  71.3500  69.1400  70.590  1662544  
2020-09-10 00:00:00-04:00  70.6750  69.0400  69.140  1650643  
2020-09-11 00:00:00-04:00  69.3500  68.1600  68.849  1865467  
2020-09-14 00:00:00-04:00  69.9000  68.3495  69.540  1408925  
2020-09-15 00:00:00-04:00  71.1500  69.8150  70.410  1606302  

[178 rows x 2520 columns]